In [5]:
import os
import shutil

In [ ]:
regions ={'a','b'}


In [6]:
# get common data files from both years

def get_common_csv_files(folder1, folder2):
    files_folder1 = set(file for file in os.listdir(folder1) if file.endswith('.csv'))
    files_folder2 = set(file for file in os.listdir(folder2) if file.endswith('.csv'))
    
    common_files = files_folder1.intersection(files_folder2)
    return common_files

def copy_files(source_folder, destination_folder, files_to_copy):
    for file in files_to_copy:
        source_path = os.path.join(source_folder, file)
        destination_path = os.path.join(destination_folder, file)
        shutil.copy2(source_path, destination_path)

# Replace these paths with your actual paths
folder1_path = 'E:/xie/Sensor Files/4. impute/full MICE imputated/2018'
folder2_path = 'E:/xie/Sensor Files/4. impute/full MICE imputated/2019'
destination_folder1_path = 'E:/xie/Sensor Files/5. dataset/full MICE imputed datasets/2018'
destination_folder2_path = 'E:/xie/Sensor Files/5. dataset/full MICE imputed datasets/2019'

common_files = get_common_csv_files(folder1_path, folder2_path)

# Copy common files to destination folders
copy_files(folder1_path, destination_folder1_path, common_files)
copy_files(folder2_path, destination_folder2_path, common_files)

print("Common CSV files copied successfully.")
